In [74]:
import pandas as pd
import seaborn as sns 
import pandas as pd
import numpy as np
from random import sample

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

## Bagging

Uma técnica que tem como objetivos aprimorar a acurácia e evitar o overfiting, reduzindo a variância. A partir dela, construimos várias vezes o mesmo modelo base para diferentes conjuntos de dados da mesma base de treinamento, obtendo uma estimativa final através da média dos resultados dos modelos base. O Bagging pode ser aplicado com diferentes algoritmos de machine learning, sendo flexível em sua escolha.

## Passo a Passo e Implementação

### Bootstrap

Temos uma base disponível. Dessa base, realizaremos uma amostragem com reposição (bootstrap) e geraremos novas bases uniformes, aleatórias e com reposição. Sendo a quantidade e tamanho das bases a ser definidas de antemão.

In [75]:
df = sns.load_dataset('penguins')
df.dropna(inplace = True)
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male


In [76]:
# exclui uma das espécies para facilitar o entendimento
df.drop(df.query('species == "Chinstrap"').index, inplace = True)
df.species.value_counts()

Adelie    146
Gentoo    119
Name: species, dtype: int64

In [77]:
alt = {'Adelie' : 0, 'Gentoo' : 1}
df.species = df.species.map(alt)
df.species.value_counts()

0    146
1    119
Name: species, dtype: int64

In [78]:
def bootstrap(base, amostras):
    subdf_X = []
    subdf_y = []
        
    for i in range(0,int(len(base)/amostras)):
        subbase = base.sample(int(len(base)/amostras))
        X = subbase.drop(['species', 'island', 'sex'], axis = 1)
        y = subbase.species
        subdf_X.append(X)
        subdf_y.append(y)
                
    return subdf_y, subdf_X

In [79]:
subdf_y, subdf_X = bootstrap(df, 35)
subdf_X[1]

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
303,50.0,15.9,224.0,5350.0
261,49.6,16.0,225.0,5700.0
106,38.6,17.2,199.0,3750.0
82,36.7,18.8,187.0,3800.0
340,46.8,14.3,215.0,4850.0
304,44.9,13.8,212.0,4750.0
136,35.6,17.5,191.0,3175.0


### Modelagem

Então escolheremos nosso modelo para classificação ou regressão, como por exemplo árveore de decisão. O modelo escolhido será aplicado para cada nova base gerada no passo anterior

In [80]:
# arvore de decisão

def modelagem(subdf_y, subdf_X):
    arvores = []
    for i in range(0,int(len(subdf_y))):
        clf = tree.DecisionTreeClassifier(random_state = 1234)
        clf = clf.fit(subdf_X[i], subdf_y[i])
        y_pred = clf.predict(subdf_X[i])
        arvores.append(y_pred)
                   
    return arvores

In [81]:
arvores = modelagem(subdf_y, subdf_X)
arvores

[array([0, 0, 0, 1, 1, 0, 1], dtype=int64),
 array([1, 1, 0, 0, 1, 1, 0], dtype=int64),
 array([0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([1, 0, 0, 0, 1, 1, 1], dtype=int64),
 array([0, 1, 0, 1, 1, 0, 0], dtype=int64),
 array([0, 0, 1, 0, 1, 0, 0], dtype=int64),
 array([0, 1, 1, 0, 0, 0, 0], dtype=int64)]

### Aggregating

Após ajustarmos os modelos na etapa anterior, combinamos os resultados por sua média (para modelos de regressão) ou por votação (para modelos de classificação).

In [82]:
df_pred = pd.DataFrame()

for i in range(0,int(len(subdf_X))):
    subdf_X[i]['original'] = subdf_y[i]
    subdf_X[i]['predito'] = arvores[i]

    for j in range(0,int(len(subdf_X))):
    
        df_pred = pd.concat([df_pred,subdf_X[j]])

df_pred.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,original,predito
42,36.0,18.5,186.0,3100.0,0.0,0.0
73,45.8,18.9,197.0,4150.0,0.0,0.0
96,38.1,18.6,190.0,3700.0,0.0,0.0
305,50.8,17.3,228.0,5600.0,1.0,1.0
284,45.8,14.2,219.0,4700.0,1.0,1.0


In [85]:
df_pred.groupby(['original', 'predito'])[['original', 'predito']].count()

,,original,predito
original,predito,,
0.0,0.0,120,120
1.0,1.0,76,76
